# Introduçãos às CNNs no PyTorch

## Objetivos

Este notebook contém exemplo numérico de uma rede com uma camada convolucional e uma camada densa. A camada convolucional possui ativação reLU e max-pooling. Já a camada densa possui uma única saída com ativação sigmóide.

Com esse exemplo, aprende-se a:
- construir a rede utilizando subclasses de Module do PyTorch
- inicializar os pesos e biases da rede convolucional e da rede densa
- visualizar os dados intermediários da rede

## Importação dos Módulos

In [1]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.functional as F

from collections import OrderedDict

np.set_printoptions(precision=3) # ponto flutuante com 3 casas para facilitar a impressão

In [2]:
# verifica se a GPU está disponível
use_gpu = torch.cuda.is_available()
print("Usando GPU:", use_gpu)

Usando GPU: False


## Definição da Rede, camadas convolucionais e densas

### Definição de camadas

Em redes neurais, uma camada é usualmente um neurônio, que inclui a soma de multiplicação de
pesos ou convolução e uma ativação usualmente não linear. Podemos dizer que o max-pooling 
também faz parte da camada.

A rede que iremos utilizar neste exemplo possui 2 camadas: uma convolucional e outra densa.
A camada convolucional terá ativação reLU e um max-pooling, já a camada densa terá uma
ativação sigmóide.

### Rede a ser implementada

<img src='../figures/RedeIntroKeras.png' width='600pt'>

### Criando redes np PyTorch

Uma CNN no PyTorch é criada utilizando uma classe que é subclasse do *torch.nn.Module*. Essa classe poderá ter variáveis que também são instâncias de subclasses de *torch.nn.Module*.

- *torch.nn.Conv2d*
- *torch.nn.ReLU*
- *torch.nn.MaxPool2d*
- *torch.nn.Linear*

Todas estas classes são também subclasses de *torch.nn.Module* e podem ser instanciadas no contrutor para serem utilizadas na classe (ver [Documentação torch.nn](http://pytorch.org/docs/master/nn.html))

### Implementação do modelo

In [3]:
class Model(nn.Module):
    
    # Definição de Modules que serão utilizados na rede
    def __init__(self):
        super(Model, self).__init__()
        
        # camada convolucional
        self.conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(2, 2), 
                              padding=0)
        self.relu1 = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        
        # camada densa
        self.dense = nn.Linear(in_features=3*4, out_features=1)
        self.relu2 = nn.ReLU()
    
    # Método de definição obrigatória
    # Sequencia que será efetivamente executada para obter a saída da rede
    def forward(self, x):
        # passa os dados pela camada convolucional
        x = self.conv(x)
        x = self.relu1(x)
        x = self.max_pool(x)
        
        # faz o flatten dos dados
        x = x.reshape(-1, 3*4)
        
        # passa os dados pela camada densa
        x = self.dense(x)
        x = self.relu2(x)
        
        return x

In [4]:
model = Model()
print(model)

Model(
  (conv): Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
  (relu1): ReLU()
  (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (dense): Linear(in_features=12, out_features=1, bias=True)
  (relu2): ReLU()
)


## Criação dos parâmetros da rede

Neste exemplo não haverá treinamento, iremos inicializar os parâmetros da rede com valores conhecidos, de acordo com a figura.

### Kernel da convolução

In [5]:
# número de filtros
n_filters = 3    

# comprimento e largura dos filtros
k_height = k_width = 2 
kernel_shape = (k_height, k_width)

W_conv = torch.FloatTensor(
                 [[[1,2],
                   [3,4]],
                  [[5,6],
                   [7,8]],
                  [[9,10],
                   [11,12]]]).reshape(n_filters,1,k_height,k_width)

### Bias da Convolução

In [6]:
# valor de bias
f_bias = 0.1     
bias_conv = torch.arange(1,n_filters+1).float() * f_bias
print("Bias da convolução:",bias_conv)

Bias da convolução: tensor([0.1000, 0.2000, 0.3000])


### Pesos para a camada densa

In [7]:
# após o max pooling, são 3 imagens 2x2 = 12
W_dense = torch.arange(12).float().reshape(1, 12)
print("Pesos da camada densa:\n",W_dense)

Pesos da camada densa:
 tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]])


### Bias para a camada densa

In [8]:
bias_dense = torch.ones(1) * f_bias
print("Bias da camanda densa:",bias_dense)

Bias da camanda densa: tensor([0.1000])


### Sumário do shape dos parâmetros

In [9]:
from IPython.display import display,HTML
# Mostra o shape dos pesos das camadas
display(HTML('<h3>Shape dos pesos</h3>'))
print('{:11} {}'.format('W_conv:', W_conv.shape))
print('{:11} {}'.format('bias_conv:', bias_conv.shape))
print('{:11} {}'.format('W_dense:', W_dense.shape))
print('{:11} {}'.format('bias_dense:', bias_dense.shape))
print()

W_conv:     torch.Size([3, 1, 2, 2])
bias_conv:  torch.Size([3])
W_dense:    torch.Size([1, 12])
bias_dense: torch.Size([1])



## Criação da rede model e carregamento dos pesos (parâmetros)

In [10]:
# cria o modelo
model = Model()

# Dicionário com os pesos para a rede
my_weights = OrderedDict([
    ('conv.weight',  W_conv),
    ('conv.bias',    bias_conv),
    ('dense.weight', W_dense),
    ('dense.bias',   bias_dense),
])

# aplica os pesos criados à rede
model.load_state_dict(my_weights)

### Visualização dos pesos da rede na forma de dicionário

In [11]:
model.state_dict()

OrderedDict([('conv.weight', tensor([[[[ 1.,  2.],
                        [ 3.,  4.]]],
              
              
                      [[[ 5.,  6.],
                        [ 7.,  8.]]],
              
              
                      [[[ 9., 10.],
                        [11., 12.]]]])),
             ('conv.bias', tensor([0.1000, 0.2000, 0.3000])),
             ('dense.weight',
              tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]])),
             ('dense.bias', tensor([0.1000]))])

### Visualização dos pesos da rede na forma de tensores

In [12]:
for W_name, W in model.state_dict().items():
    display(HTML('<h4>{}</h4>'.format(W_name)))
    print(W)

tensor([[[[ 1.,  2.],
          [ 3.,  4.]]],


        [[[ 5.,  6.],
          [ 7.,  8.]]],


        [[[ 9., 10.],
          [11., 12.]]]])


tensor([0.1000, 0.2000, 0.3000])


tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]])


tensor([0.1000])


## Predição da Rede e Visualização dos tensores em cada camada

### Imagem de Entrada: Variable (1,1,5,6) uma amostra, um canal, 5 linhas e 6 colunas

In [13]:
X = torch.FloatTensor(
             [[[[0,0,0,0, 0,0],
                [0,0,1,0, 0,0],
                [0,0,0,0, 0,0],
                [0,0,0,0,-1,0],
                [0,0,0,0, 0,0]]]])

display(HTML('<h3>Entrada na rede</h3>'))
display(HTML('<h4>X</h4>'))
print(X)

tensor([[[[ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]]]])


## Predição

### Mostra as saídas de cada camada da rede

In [14]:
for name, layer in model.named_children():
    display(HTML('<h4>{}:{}</h4>'.format(name, X.shape)))
    if name == 'dense': X = X.reshape(-1, 3*4)
    X = layer(X)
    print(X)

tensor([[[[  0.1000,   4.1000,   3.1000,   0.1000,   0.1000],
          [  0.1000,   2.1000,   1.1000,   0.1000,   0.1000],
          [  0.1000,   0.1000,   0.1000,  -3.9000,  -2.9000],
          [  0.1000,   0.1000,   0.1000,  -1.9000,  -0.9000]],

         [[  0.2000,   8.2000,   7.2000,   0.2000,   0.2000],
          [  0.2000,   6.2000,   5.2000,   0.2000,   0.2000],
          [  0.2000,   0.2000,   0.2000,  -7.8000,  -6.8000],
          [  0.2000,   0.2000,   0.2000,  -5.8000,  -4.8000]],

         [[  0.3000,  12.3000,  11.3000,   0.3000,   0.3000],
          [  0.3000,  10.3000,   9.3000,   0.3000,   0.3000],
          [  0.3000,   0.3000,   0.3000, -11.7000, -10.7000],
          [  0.3000,   0.3000,   0.3000,  -9.7000,  -8.7000]]]],
       grad_fn=<ThnnConv2DBackward>)


tensor([[[[ 0.1000,  4.1000,  3.1000,  0.1000,  0.1000],
          [ 0.1000,  2.1000,  1.1000,  0.1000,  0.1000],
          [ 0.1000,  0.1000,  0.1000,  0.0000,  0.0000],
          [ 0.1000,  0.1000,  0.1000,  0.0000,  0.0000]],

         [[ 0.2000,  8.2000,  7.2000,  0.2000,  0.2000],
          [ 0.2000,  6.2000,  5.2000,  0.2000,  0.2000],
          [ 0.2000,  0.2000,  0.2000,  0.0000,  0.0000],
          [ 0.2000,  0.2000,  0.2000,  0.0000,  0.0000]],

         [[ 0.3000, 12.3000, 11.3000,  0.3000,  0.3000],
          [ 0.3000, 10.3000,  9.3000,  0.3000,  0.3000],
          [ 0.3000,  0.3000,  0.3000,  0.0000,  0.0000],
          [ 0.3000,  0.3000,  0.3000,  0.0000,  0.0000]]]],
       grad_fn=<ThresholdBackward0>)


tensor([[[[ 4.1000,  3.1000],
          [ 0.1000,  0.1000]],

         [[ 8.2000,  7.2000],
          [ 0.2000,  0.2000]],

         [[12.3000, 11.3000],
          [ 0.3000,  0.3000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)


tensor([[281.5000]], grad_fn=<ThAddmmBackward>)


tensor([[281.5000]], grad_fn=<ThresholdBackward0>)


# Sugestões de atividades

1. Quais as vantagens de utilizar Sequential na definição do modelo?
2. Como calcular o número de parâmetros da rede?
3. Retirar o bias e recalcular o número de parâmetros a serem treinados
4. A rede foi projetada para aceitar entrada com shape (1,5,6). Mudar a rede para aceitar:
   a) entrada com shape (1,6,6)
   b) entrada com shape (3,6,6)
5. Inserir mais uma camada convolucional com 4 filtros de saída e kernel (3,3)

# Referências

- [PyTorch Documentation](http://pytorch.org/docs/master/index.html)


# Aprendizados com este notebook
